In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
# mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
# clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
# landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
# landsat_vis = {
#     'min': 0,
#     'max': 1,
#     'bands': ['b5', 'b4', 'b3']
# }
# Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
# Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
# Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
# Map

In [5]:
countries = ee.FeatureCollection('users/giswqs/public/countries')
Map.addLayer(countries, {}, 'countries')


In [6]:
roi = countries.filter(ee.Filter.eq('id', 'CHL'))
Map.addLayer(roi, {}, 'roi')

In [7]:
work_dir = os.path.expanduser('../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')

In [8]:
start_date = '2019-12-01'
end_date = '2021-03-31'

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [9]:
median = l8.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map.centerObject(patagonia_grande, 10)
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [10]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [11]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [12]:
def addMonth(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('M'))
    return image.addBands(ee.Image(img_doy).rename('month').toInt())

In [13]:
def addDOY(image):
    img_date = ee.Date(image.date())
    img_doy = ee.Number.parse(img_date.format('D'))
    return image.addBands(ee.Image(img_doy).rename('doy').toInt())

In [14]:
withNDVI = l8.map(addNDVI).map(addDate).map(addMonth).map(addDOY)

In [15]:
greenest = withNDVI.qualityMosaic('NDVI').clip(patagonia_grande)

In [16]:
greenest.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

In [17]:
ndvi = greenest.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map.addLayer(ndvi, {'palette': palette}, 'NDVI')

In [18]:
Map.addLayer(greenest, visParams, 'Greenest pixel')
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [19]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[-49.845576304360975, -69.3669060002117], controls=(WidgetControl(options=['position'], widget=HBox…

In [20]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]
# paleta de 18 colores


In [21]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [22]:
landcover_2018.bandNames().getInfo()

['b1']

In [23]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [24]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [25]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [26]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [27]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [28]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [29]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [30]:
# poligono = ee.FeatureCollection([
#     ee.Feature(achaparrado),
#     ee.Feature(agricolas),
#     ee.Feature(agua),
#     ee.Feature(arenalroca),
#     ee.Feature(bosquedegradado),
#     ee.Feature(cuncuna),
#     ee.Feature(estepa),
#     ee.Feature(humedal),
#     ee.Feature(mallin),
#     ee.Feature(matorralabierto),
#     ee.Feature(matdenso),
#     ee.Feature(plantacion1),
#     ee.Feature(plantacion2),
#     ee.Feature(plantacion3),
#     ee.Feature(pradera),
#     ee.Feature(primario),
#     ee.Feature(secundario),
#     ee.Feature(sombra),
#     ee.Feature(suelo),    
# ])


In [31]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [32]:
dataset = greenest.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [33]:
dataset.size().getInfo()

EEException: User memory limit exceeded.

In [ ]:
print(dataset.first().getInfo())

In [ ]:
# Guardamos las bandas
bands = ['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'BQA',
 'NDVI',
 'date',
 'month',
 'doy']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [ ]:
# print(training.first().getInfo())

In [ ]:
# print(validation.first().getInfo())

In [ ]:
classifier = ee.Classifier.smileRandomForest(100).train(dataset, label, bands)

In [ ]:
# Clasificando con las mismas bandas de entrenamiento
result = greenest.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classificado')
Map

In [ ]:
result.getInfo()

In [ ]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [ ]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [ ]:
land = result.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcoverf, {}, 'Land cover Random Forest Región')
legend_dict = {
    '01 Achaparrado': '22c129',
    '02 Agrícola': 'f0ff00',
    '03 Agua': '0c40f1',
    '04 Arenal o Roca': '685205',
    '05 Bosque Degradado': '0d4404',
    '06 Cuncunas': 'e82d13',
    '07 Estepa': 'f99504',
    '08 Humedal': '17d4c6',
    '09 Mallin': '1c6330',
    '10 Matorral abierto': '24ff00',
    '11 Matorral denso': '2de50f',
    '12 Plantaciones': 'ccba7d',
    '13 Praderas': 'dec61a',
    '14 Primario': '28d909',
    '15 Ríos': '03c2ff',
    '16 Secundario': '4e8c44',
    '17 Sombra topo': 'fdfdfd',
    '18 Suelo desnudo': 'c0c233'
}
Map.add_legend(legend_title="Random Forest Patagonia", legend_dict=legend_dict)
Map

In [ ]:
# train_accuracy = classifier.confusionMatrix()

In [ ]:
# train_accuracy.getInfo()

In [ ]:
# train_accuracy.accuracy().getInfo()

In [ ]:
# train_accuracy.kappa().getInfo()

In [ ]:
# train_accuracy.producersAccuracy().getInfo()

In [ ]:
# train_accuracy.consumersAccuracy().getInfo()

In [ ]:
# validacion = datasetval.classify(classifier)

In [ ]:
# testAccuracy = validacion.errorMatrix('ID', 'classification')

In [ ]:
# testAccuracy.accuracy().getInfo()

In [ ]:
# testAccuracy.getInfo()